In [2]:
import scrapy
from scrapy import Selector
import requests
import pandas as pd
import time

pd.set_option('display.width', 1500)

In [17]:
def get_listings():
    url = 'https://www.wg-gesucht.de//1-zimmer-wohnungen-und-wohnungen-in-Berlin.8.1+2.1.0.html'
    html = requests.get(url).text
    sel = Selector(text = html)
    listings_list = list(sel.xpath('//*[contains(@class,"wgg_card offer_list_item  ")]/div/div/a/@href').extract())
    listings_list = list(set(listings_list))
    return [listing for listing in listings_list if listing.endswith('html')]

In [ ]:
listings_list = get_listings()

In [4]:
def scrape_wg_gesucht_data(input_list):
    """Function for scraping data from each individual listing."""
    appended_data = []
    counter = 0
    for listing in input_list:
        try:
            print(listing)
            #try:
            listing_html = 'https://www.wg-gesucht.de/'+listing
            html = requests.get(listing_html).text
            sel = Selector(text = html)
            apartment_dict = {}
            print('listing...', end = ' ')
            apartment_dict['url'] = listing
            #1. listing size
            print('size...', end = ' ')
            apartment_dict['size'] = sel.xpath('//*[contains(@class,"headline headline-key-facts")]/text()').extract()[0].strip() 
            #2. rent
            print('rent...', end = ' ')
            apartment_dict['rent'] = sel.xpath('//*[contains(@class,"headline headline-key-facts")]/text()').extract()[2].strip()   
            #3. rooms
            print('rooms...', end = ' ')
            if len(sel.xpath('//*[contains(@class,"headline headline-key-facts")]/text()').extract()) == 6:
                apartment_dict['rooms'] = sel.xpath('//*[contains(@class,"headline headline-key-facts")]/text()').extract()[4].strip()
            else:
                pass
            #4. floor
            #print('floor...', end = ' ')
            #raw_floor = sel.xpath('//*[contains(@class,"col-xs-6 col-sm-4 text-center print_text_left")]/text()').extract()[2]
            #apartment_dict['floor'] = ''.join(raw_floor.split())
            #5. deposit
            print('deposit...', end = ' ')
            if sel.xpath('//*[contains(@id,"kaution")]/@value') != []:
                apartment_dict['deposit'] = sel.xpath('//*[contains(@id,"kaution")]/@value').extract()[0]
            else:
                pass
            #6. address_street
            print('street...', end = ' ')
            raw_street = sel.xpath('//*[contains(@class,"col-sm-4 mb10")]/a/text()').extract()[0]
            apartment_dict['street'] = ' '.join(raw_street.split())
            #7. district
            print('location...', end = ' ')
            raw_district = sel.xpath('//*[contains(@class,"col-sm-4 mb10")]/a/text()').extract()[1]
            apartment_dict['location'] = ' '.join(raw_district.split())
            #8. available_from
            print('available from...', end = ' ')
            apartment_dict['available_from'] = sel.xpath('//*[contains(@class,"col-sm-3")]/p/b/text()').extract()[0]
            #9. term
            #available_until
            print('available until...')
            if 'frei bis:' in [item.strip() for item in sel.xpath('//*[contains(@class,"col-sm-3")]/p/text()').extract()]:
                output_term = sel.xpath('//*[contains(@class,"col-sm-3")]/p/b/text()').extract()[1]
            else:
                output_term = 'unlimited'
            apartment_dict['available_until'] = output_term
            #10. tausch
            if 'Tauschangebot' in [item.strip() for item in sel.xpath('//*[contains(@class,"col-sm-3")]/p/text()').extract()]:
                tausch = True
            else:
                tausch = False
            apartment_dict['tausch'] = tausch
            appended_data.append(apartment_dict)
            counter = counter+1
            if counter%10 == 0:
                print('Aggregated listings: {}.'.format(counter), end = ' ')
                time.sleep(10)
        except:
            listing+' passed!'
            pass
    df = pd.DataFrame(appended_data)
    df = df.reset_index().rename(columns = {'index':'id'})
    return df

In [5]:
final_df = scrape_wg_gesucht_data(listings_list)

/wohnungen-in-Berlin-Moabit.9446996.html
listing... size... rent... rooms... deposit... street... location... available from... available until...
/wohnungen-in-Berlin-Prenzlauer-Berg.9274117.html
listing... size... rent... rooms... deposit... street... location... available from... available until...
/wohnungen-in-Berlin-Weissensee.9200978.html
listing... size... rent... rooms... deposit... street... location... available from... available until...
/wohnungen-in-Berlin-Prenzlauer-Berg.9050768.html
listing... size... rent... rooms... deposit... street... location... available from... available until...
/wohnungen-in-Berlin-Neukoelln.8754115.html
listing... size... rent... rooms... deposit... street... location... available from... available until...
/wohnungen-in-Berlin-Neukoelln.8959085.html
listing... size... rent... rooms... deposit... street... location... available from... available until...
/wohnungen-in-Berlin-Neukoelln.8018652.html
listing... size... rent... rooms... deposit...

In [6]:
final_df

,id,url,size,rent,rooms,deposit,street,location,available_from,available_until,tausch
0,0,/wohnungen-in-Berlin-Moabit.9446996.html,60m²,1250€,2,400,Rathenower Str 22,10559 Berlin Moabit,16.08.2022,23.09.2022,False
1,1,/wohnungen-in-Berlin-Prenzlauer-Berg.9274117.html,42m²,60€,"2,5",100,Hufelandstrasse,10407 Berlin Prenzlauer Berg,21.07.2022,31.07.2022,False
2,2,/wohnungen-in-Berlin-Weissensee.9200978.html,75m²,1700€,"2,5",3600,Antonplatz,13088 Berlin Weißensee,22.07.2022,30.07.2023,False
3,3,/wohnungen-in-Berlin-Prenzlauer-Berg.9050768.html,62m²,550€,2,500,Senefelderstraße,10437 Berlin Prenzlauer Berg,25.07.2022,31.07.2022,False
4,4,/wohnungen-in-Berlin-Neukoelln.8754115.html,72m²,300€,3,500,Braunschweiger Straße,12055 Berlin Neukölln,13.07.2022,17.07.2022,False
5,5,/wohnungen-in-Berlin-Neukoelln.8959085.html,62m²,1100€,2,500,Silbersteinstrasse 139,12051 Berlin Neukölln,15.08.2022,15.09.2022,False
6,6,/wohnungen-in-Berlin-Neukoelln.8018652.html,110m²,700€,3,NaN,Weserstraße,12059 Berlin Neukölln,22.07.2022,12.08.2022,False
7,7,/wohnungen-in-Berlin-Lankwitz.9452236.html,64m²,692€,2,692,Mudrastraße 29,12249 Berlin Lankwitz,14.08.2022,16.01.2023,False
8,8,/wohnungen-in-Berlin-Pankow.9164993.html,57m²,900€,2,150,Körnerstr.48,13156 Berlin Pankow,14.07.2022,30.09.2022,False
9,9,/wohnungen-in-Berlin-Charlottenburg.9455770.html,50m²,850€,2,700,Bismarck,10627 Berlin Charlottenburg,25.07.2022,31.08.2022,False


In [7]:
final_df['size'] = [int(size.strip('m²')) for size in final_df['size']]
final_df['rent'] = [int(rent.strip('€')) for rent in final_df['rent']]
final_df['district'] = [location.split(' ')[-1] for location in final_df['location']]
final_df['zipcode'] = [location.split(' ')[0] for location in final_df['location']]

In [16]:
filtered_df = final_df[(final_df['rent'] < 1500) & (final_df['available_until'] == 'unlimited') & (final_df['tausch'] == False)]
filtered_df = filtered_df[['url','size','rent','rooms','deposit','street','district','zipcode','available_from']]
filtered_df

,url,size,rent,rooms,deposit,street,district,zipcode,available_from
15,/wohnungen-in-Berlin-Mitte.9440359.html,18,450,1,500,Lichtenberger Staße 2,Mitte,10179,01.08.2022
17,/1-zimmer-wohnungen-in-Berlin-Steglitz-Zehlend...,26,450,NaN,400,Mühlenstraße 57,Steglitz-Zehlendorf,12249,10.07.2022
